# 2. Association rules

Rules time! And time to reckon with the decisions I made for how to store data.

I'm riffing on the usual shopping basket approach -- I'll be treating each act as a receipt, and each speaker as an item.

---

## Imports

In [15]:
import json
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter, defaultdict

with open('speakers.json', 'r') as f:
    d = json.load(f)

# h/t chatgpt I did not want to type all this out again.
records = []
for episode, content in d.items():
    for segment, counts in content.items():
        record = {'segment': segment}
        record.update(counts)
        records.append(record)

speaker_df = pd.DataFrame(records).fillna(0).set_index('segment').astype(int)

In [10]:
# pd.DataFrame([d[each] for each in d])

In [2]:
eps = list(d.keys())

In [3]:
d['404.txt'].keys()

dict_keys(['404_teaser', '404_act_1', '404_act_2', '404_act_3', '404_act_4', '404_act_5'])

In [31]:
'GARAK' in d['404.txt']['404_teaser']

True

In [32]:
# for ep in d:
#     for act in d[ep]:
#         print(d[ep][act])

In [33]:
for ep in d:
    for act in d[ep]:
        if 'GARAK,' in d[ep][act].keys():
            print(d[ep][act].keys())

### could...

loop over dictionaries, make sets per episode of characters who show up, then count how often they show up in each episode set

for characters like MULLIBOK and OLD JAKE -- helps me identify one-offs with lots of dialog

In [34]:
ep_ds = []
for k, v in d.items():
    ep_ds.append([v for k, v in d[k].items()])

In [35]:
def speakers_from_ep(ep_list):
    ep_speakers = []
    for each in ep_list:
        ep_speakers.extend(list(each.keys()))
    return ep_speakers

In [36]:
ep_speaker_lists = [speakers_from_ep(each) for each in ep_ds]

In [37]:
ep_speaker_counters = [Counter(each) for each in ep_speaker_lists]

In [38]:
[each for each in ep_speaker_counters if 'GARAK, ' in each.keys()]

[]

In [39]:
ep_speaker_sums = defaultdict(int)
for counter in ep_speaker_counters:
    for k, v in counter.items():
        if (',' in k) and (len(k) > 5):
            print(k)
        ep_speaker_sums[k] += 1

In [40]:
speaker_act_appearances = pd.Series(ep_speaker_sums)
speaker_act_appearances = speaker_act_appearances.sort_values(ascending=False)

in this step I discovered:
- ("O'BRIEN", "0'BRIEN")
- empty strings (replaced with "blank")

- I should definitely just be upper-casing everything
definitely some further work to do with matching names... VIC/Vic/Fontaine...

In [51]:
speaker_act_appearances[speaker_act_appearances.index.str.lower().str.contains('vic')]

VIC    8
Vic    3
dtype: int64

In [52]:
speaker_act_appearances[speaker_act_appearances.index.str.lower().str.contains('font')]

Series([], dtype: int64)

In [ ]:
# just search speakers.json for this
# GARAX????????????
# in 492_act_5

# KIRA Come in. 
# WEYOUN I'm glad.

# GAPAK in 497_act_4

In [ ]:
# GARAX, GAPAX

In [ ]:
# write to csv later

In [63]:
speaker_act_appearances[300:325]

JADZIA                  1
ODO WORF AND O'BRIEN    1
DRUNKEN KLINGON         1
TOWERING KLINGON        1
RICHARD                 1
HEAD GUARD              1
GINT                    1
VIRA-KIKARA             1
(agreeing)              1
ATTENDANT               1
ANSLEM                  1
by Jake Sisko           1
BRATHAW                 1
OPERATIVE               1
NORVA                   1
INFIRMARY COM VOICE     1
JESSICA                 1
EPRAN                   1
BASHIR AND O'BRIEN      1
(To Toman'torax)        1
PATIENT                 1
TREVEAN                 1
EKORIA                  1
LATIA                   1
OMET'IKLAN              1
dtype: int64

In [16]:
speaker_act_appearances.index[5], speaker_act_appearances.index[50]

("O'BRIEN", "0'BRIEN")

In [17]:
speaker_act_appearances.index[23] == ''

True

In [18]:
speaker_act_appearances[50:100]

0'BRIEN              4
TAIN                 4
LWAXANA              3
WOMAN                3
CUT TO:              3
KOR                  3
KLINGON              3
ROMULAN              3
MILA                 3
SLOAN                3
SHAKAAR              3
Vic                  3
ODO ALIEN            3
MALE VOICE           3
FUREL                3
JAKE ALIEN           3
RUSOT                3
SOLBOR               3
SARAH PROPHET        3
SISKO ALIEN          3
MUNIZ                3
JENNIFER             3
JARO                 3
JEM'HADAR SOLDIER    3
CRETAK               3
BAJORAN OFFICER      3
LI                   3
CARDASSIAN           3
FEMALE COM VOICE     3
FALA                 2
INTENDANT            2
IKAT'IKA             2
MAN                  2
SIRELLA              2
OPAKA                2
JACK                 2
WHATLEY              2
BENNY                2
DEPUTY               2
SECURITY OFFICER     2
INTERCUT:            2
MORA                 2
B.C.                 2
HAZAR      

In [19]:
speaker_ds = []

for k, v in d.items():
    for act, speakers in v.items():
        speaker_ds.append(speakers)

In [20]:
from collections import defaultdict

## identifying one-off and very minor characters
- if a character has fewer than 

In [21]:
all_speakers_dict = defaultdict(int)
for d in speaker_ds:
    for k, v in d.items():
        all_speakers_dict[k] += v

In [22]:
speakers = pd.Series(all_speakers_dict)

In [23]:
sorted_speakers = speakers.sort_values(ascending=False).index

In [24]:
speakers.sort_values(ascending=False)

SISKO                                9398
KIRA                                 5846
BASHIR                               5497
O'BRIEN                              5276
QUARK                                5116
                                     ... 
TROOPER                                 1
(FORMERLY SCENE 35)                     1
OM                                      1
KUDKA'ETAN                              1
DISSOLVE INTO A MONTAGE SEQUENCE:       1
Length: 911, dtype: int64

In [25]:
sorted_speakers[:75]

Index(['SISKO', 'KIRA', 'BASHIR', 'O'BRIEN', 'QUARK', 'ODO', 'DAX', 'WORF',
       'GARAK', 'JAKE', 'NOG', 'ROM', 'DUKAT', 'EZRI', 'WINN', 'WEYOUN',
       'MARTOK', 'KEIKO', 'DAMAR', 'ZEK', 'KASIDY', 'VIC', 'BAREIL',
       'COMPUTER VOICE', 'EDDINGTON', 'FEMALE SHAPE-SHIFTER', 'KOR', 'SLOAN',
       'JOSEPH', 'ISHKA', 'BRUNT', 'LEETA', 'GUL DUKAT', 'ROSS', 'LWAXANA',
       'MORA', 'BILBY', 'ZIYAL', 'JACK', 'SHAKAAR', 'TAIN', 'ARISSA', 'GHEMOR',
       'JENNIFER', 'MELORA', 'CH'POK', 'NATIMA', 'GOWRON', 'MARTUS', 'PEL',
       'JORAN', 'ARJIN', 'BENNY', 'MULLIBOK', 'CHANGELING', 'RIKER', 'GAILA',
       'OLD JAKE', 'VASH', 'LEYTON', 'INTENDANT', 'CRODEN', 'HUDSON', 'HANEEK',
       'MARRITZA', 'LENARA', 'B.C.', 'DERAL', 'WATTERS', 'MOLLY', 'TOSK',
       'LISA', 'GORAN'AGAR', 'GRILKA', 'LI'],
      dtype='object')

In [26]:
speaker_sets[:5]

NameError: name 'speaker_sets' is not defined

In [ ]:
# d_subset = [v for k, v in d.items()]
# d_subset = [list(each.values()) for each in d_subset[:3]]
# d_subset[0]

In [ ]:
# def speakers_from_ep(d, append_to):
#     ep_speakers = []
#     # accepts a dictionary, we get back a simple speakers list.
#     # i might want to filter these first, but let's worry about that later.
#     for key in d:
#         act_speakers = list(d[key].keys())
#         append_to.append(act_speakers)

In [113]:
top_50 = set(sorted_speakers[:50])

In [115]:
items = []

for ep in d:
    # print(f'{ep}')
    for act in d[ep]:
        # print(f'{act}')
        act_speakers = set(d[ep][act].keys())
        act_speakers = act_speakers.intersection(top_50)
        items.append(act_speakers)
    # print()

In [116]:
sum(['SISKO' in i for i in items])

729

In [117]:
speaker_occurences = {}

for each_speaker in top_50:
    speaker_occurences[each_speaker] = sum(each_speaker in i for i in items)

In [119]:
import itertools

In [126]:
def check_multi_membership(members, item):
    return all(m in item for m in members)

In [136]:
sum(check_multi_membership(['JAKE', 'SISKO'], i) for i in items)

180

In [137]:
pairs = itertools.combinations(top_50, 2)
pair_occurences = {}

for pair in pairs:
    pair_occurences[pair] = sum(check_multi_membership(pair, i) for i in items)

In [146]:
# https://www.freecodecamp.org/news/sort-dictionary-by-value-in-python/#howtosortadictionarywiththesortedmethod
sorted(pair_occurences.items(), key=lambda x:x[1], reverse=True)[:25]

[(('SISKO', 'KIRA'), 512),
 (('SISKO', "O'BRIEN"), 459),
 (('SISKO', 'BASHIR'), 454),
 (('SISKO', 'DAX'), 438),
 (('SISKO', 'ODO'), 407),
 (('KIRA', "O'BRIEN"), 407),
 (('BASHIR', "O'BRIEN"), 399),
 (('BASHIR', 'KIRA'), 382),
 (('KIRA', 'DAX'), 377),
 (('KIRA', 'ODO'), 375),
 (('DAX', "O'BRIEN"), 352),
 (('BASHIR', 'DAX'), 338),
 (('BASHIR', 'ODO'), 325),
 (('ODO', "O'BRIEN"), 317),
 (('ODO', 'DAX'), 285),
 (('SISKO', 'QUARK'), 277),
 (('QUARK', 'KIRA'), 258),
 (('QUARK', 'ODO'), 247),
 (('BASHIR', 'QUARK'), 238),
 (('QUARK', "O'BRIEN"), 236),
 (('SISKO', 'WORF'), 231),
 (('QUARK', 'DAX'), 219),
 (('KIRA', 'WORF'), 201),
 (('WORF', "O'BRIEN"), 198),
 (('BASHIR', 'WORF'), 193)]

In [ ]:
# https://www-users.cse.umn.edu/~kumar001/dmbook/dmslides/chap6_basic_association_analysis.pdf

In [147]:
triads = itertools.combinations(top_50, 3)
triad_occurences = {}

for triad in triads:
    triad_occurences[triad] = sum(check_multi_membership(triad, i) for i in items)

In [149]:
sorted(triad_occurences.items(), key=lambda x:x[1], reverse=True)[:50]

[(('SISKO', 'KIRA', "O'BRIEN"), 361),
 (('SISKO', 'KIRA', 'DAX'), 339),
 (('SISKO', 'BASHIR', 'KIRA'), 338),
 (('SISKO', 'BASHIR', "O'BRIEN"), 326),
 (('SISKO', 'DAX', "O'BRIEN"), 318),
 (('SISKO', 'KIRA', 'ODO'), 315),
 (('SISKO', 'BASHIR', 'DAX'), 297),
 (('BASHIR', 'KIRA', "O'BRIEN"), 292),
 (('KIRA', 'DAX', "O'BRIEN"), 283),
 (('SISKO', 'BASHIR', 'ODO'), 279),
 (('SISKO', 'ODO', "O'BRIEN"), 267),
 (('SISKO', 'ODO', 'DAX'), 255),
 (('BASHIR', 'KIRA', 'ODO'), 253),
 (('BASHIR', 'KIRA', 'DAX'), 253),
 (('BASHIR', 'DAX', "O'BRIEN"), 248),
 (('KIRA', 'ODO', "O'BRIEN"), 246),
 (('BASHIR', 'ODO', "O'BRIEN"), 234),
 (('KIRA', 'ODO', 'DAX'), 229),
 (('SISKO', 'QUARK', 'KIRA'), 209),
 (('BASHIR', 'ODO', 'DAX'), 200),
 (('ODO', 'DAX', "O'BRIEN"), 198),
 (('SISKO', 'QUARK', 'ODO'), 181),
 (('SISKO', 'QUARK', "O'BRIEN"), 179),
 (('SISKO', 'BASHIR', 'QUARK'), 177),
 (('QUARK', 'KIRA', 'ODO'), 177),
 (('QUARK', 'KIRA', "O'BRIEN"), 176),
 (('SISKO', 'QUARK', 'DAX'), 174),
 (('BASHIR', 'QUARK', "O'

In [152]:
# fours = itertools.combinations(top_50, 4)
# fours_occurences = {}

# for four in fours:
#     fours_occurences[four] = sum(check_multi_membership(four, i) for i in items)
    
# sorted(fours_occurences.items(), key=lambda x:x[1], reverse=True)[:50]

In [ ]:
pair_occurences, speaker_occurences

In [154]:
def create_rules_from_counts(pair_counts, speaker_counts):
    rules = {}
    for (a, b), count in pair_counts.items():
        conf_ab = count / speaker_counts[a]
        conf_ba = count / speaker_counts[b]
        rules[(a, b)] = conf_ab
        rules[(b, a)] = conf_ba
    return rules

rules = create_rules_from_counts(pair_occurences, speaker_occurences)

In [160]:
def filter_rules_by_confidence(rules, threshold):
    return {r:c for r,c in rules.items() if c > threshold}

filtered_rules = filter_rules_by_confidence(rules, .3)
sorted(filtered_rules.items(), key=lambda x:x[1], reverse=True)

[(('MORA', 'ODO'), 1.0),
 (('GHEMOR', 'KIRA'), 1.0),
 (('MELORA', 'BASHIR'), 1.0),
 (('MELORA', 'DAX'), 1.0),
 (('TAIN', 'GARAK'), 1.0),
 (('NATIMA', 'GARAK'), 1.0),
 (('JENNIFER', 'SISKO'), 1.0),
 (("CH'POK", 'SISKO'), 1.0),
 (('JOSEPH', 'SISKO'), 1.0),
 (('LWAXANA', 'ODO'), 1.0),
 (('JACK', 'BASHIR'), 1.0),
 (('PEL', 'ROM'), 1.0),
 (('BAREIL', 'KIRA'), 1.0),
 (('SLOAN', 'BASHIR'), 1.0),
 (('ISHKA', 'QUARK'), 1.0),
 (('NATIMA', 'QUARK'), 1.0),
 (('BILBY', "O'BRIEN"), 1.0),
 (("CH'POK", 'WORF'), 1.0),
 (('ZEK', 'QUARK'), 1.0),
 (('PEL', 'QUARK'), 1.0),
 (('BRUNT', 'QUARK'), 1.0),
 (('PEL', 'ZEK'), 1.0),
 (('ARISSA', 'ODO'), 1.0),
 (('KEIKO', "O'BRIEN"), 0.9672131147540983),
 (('GUL DUKAT', 'SISKO'), 0.9411764705882353),
 (('ISHKA', 'ROM'), 0.9375),
 (('ZEK', 'ROM'), 0.9375),
 (('KASIDY', 'SISKO'), 0.9347826086956522),
 (('SHAKAAR', 'KIRA'), 0.9285714285714286),
 (('BRUNT', 'ROM'), 0.9259259259259259),
 (('EDDINGTON', 'SISKO'), 0.918918918918919),
 (('JACK', "O'BRIEN"), 0.91666666666666

In [166]:
# [each for each in filtered_rules if 'GARAK' in each]

In [165]:
{k: v for k, v in filtered_rules.items() if 'GARAK' in k}

{('GOWRON', 'GARAK'): 0.5,
 ('GARAK', 'SISKO'): 0.7610062893081762,
 ('ZIYAL', 'GARAK'): 0.5185185185185185,
 ('JENNIFER', 'GARAK'): 0.5454545454545454,
 ('TAIN', 'GARAK'): 1.0,
 ('GARAK', 'BASHIR'): 0.7044025157232704,
 ('DAMAR', 'GARAK'): 0.4084507042253521,
 ('FEMALE SHAPE-SHIFTER', 'GARAK'): 0.5333333333333333,
 ('NATIMA', 'GARAK'): 1.0,
 ('GARAK', 'QUARK'): 0.31446540880503143,
 ('EDDINGTON', 'GARAK'): 0.40540540540540543,
 ('GARAK', 'KIRA'): 0.6540880503144654,
 ('GARAK', 'WORF'): 0.4025157232704403,
 ('GARAK', 'ODO'): 0.5471698113207547,
 ('MARTOK', 'GARAK'): 0.3424657534246575,
 ('GARAK', 'DAX'): 0.49056603773584906,
 ('GARAK', "O'BRIEN"): 0.5911949685534591}

In [167]:
{k: v for k, v in filtered_rules.items() if 'ODO' in k}

{('MORA', 'ODO'): 1.0,
 ('ROSS', 'ODO'): 0.5588235294117647,
 ('GOWRON', 'ODO'): 0.625,
 ('JAKE', 'ODO'): 0.580952380952381,
 ('GHEMOR', 'ODO'): 0.5,
 ('MELORA', 'ODO'): 0.5,
 ('WINN', 'ODO'): 0.4230769230769231,
 ('DUKAT', 'ODO'): 0.40384615384615385,
 ('LEETA', 'ODO'): 0.3617021276595745,
 ('WEYOUN', 'ODO'): 0.4583333333333333,
 ('GARAK', 'ODO'): 0.5471698113207547,
 ('SISKO', 'ODO'): 0.5582990397805213,
 ('ODO', 'SISKO'): 0.7796934865900383,
 ('ZIYAL', 'ODO'): 0.37037037037037035,
 ('LWAXANA', 'ODO'): 1.0,
 ('JACK', 'ODO'): 0.3333333333333333,
 ('ROM', 'ODO'): 0.4626865671641791,
 ('NOG', 'ODO'): 0.4967741935483871,
 ('EZRI', 'ODO'): 0.40625,
 ('TAIN', 'ODO'): 0.6,
 ('BAREIL', 'ODO'): 0.6666666666666666,
 ('BASHIR', 'ODO'): 0.5622837370242214,
 ('ODO', 'BASHIR'): 0.6226053639846744,
 ('DAMAR', 'ODO'): 0.4507042253521127,
 ('FEMALE SHAPE-SHIFTER', 'ODO'): 0.7,
 ('NATIMA', 'ODO'): 0.6666666666666666,
 ('KASIDY', 'ODO'): 0.5434782608695652,
 ('COMPUTER VOICE', 'ODO'): 0.618181818181818

In [169]:
{k: v for k, v in filtered_rules.items() if 'COMPUTER VOICE' in k}

{('COMPUTER VOICE', 'JAKE'): 0.3090909090909091,
 ('COMPUTER VOICE', 'SISKO'): 0.7818181818181819,
 ('LWAXANA', 'COMPUTER VOICE'): 0.3333333333333333,
 ('COMPUTER VOICE', 'BASHIR'): 0.6545454545454545,
 ('COMPUTER VOICE', 'QUARK'): 0.43636363636363634,
 ('COMPUTER VOICE', 'KIRA'): 0.6818181818181818,
 ('COMPUTER VOICE', 'ODO'): 0.6181818181818182,
 ('COMPUTER VOICE', 'DAX'): 0.6272727272727273,
 ('COMPUTER VOICE', "O'BRIEN"): 0.6363636363636364}

In [170]:
{k: v for k, v in filtered_rules.items() if 'BASHIR' in k}

{('MORA', 'BASHIR'): 0.5,
 ('ROSS', 'BASHIR'): 0.6764705882352942,
 ('GOWRON', 'BASHIR'): 0.8125,
 ('JAKE', 'BASHIR'): 0.5523809523809524,
 ('GHEMOR', 'BASHIR'): 0.5,
 ('MELORA', 'BASHIR'): 1.0,
 ('WINN', 'BASHIR'): 0.5192307692307693,
 ('DUKAT', 'BASHIR'): 0.4807692307692308,
 ('LEETA', 'BASHIR'): 0.723404255319149,
 ('WEYOUN', 'BASHIR'): 0.5277777777777778,
 ('GARAK', 'BASHIR'): 0.7044025157232704,
 ('SISKO', 'BASHIR'): 0.6227709190672154,
 ('BASHIR', 'SISKO'): 0.7854671280276817,
 ('ZIYAL', 'BASHIR'): 0.4074074074074074,
 ('LWAXANA', 'BASHIR'): 0.6111111111111112,
 ('JACK', 'BASHIR'): 1.0,
 ('ROM', 'BASHIR'): 0.48507462686567165,
 ('NOG', 'BASHIR'): 0.5096774193548387,
 ('EZRI', 'BASHIR'): 0.65625,
 ('JENNIFER', 'BASHIR'): 0.5454545454545454,
 ('TAIN', 'BASHIR'): 0.7,
 ('BAREIL', 'BASHIR'): 0.4074074074074074,
 ('DAMAR', 'BASHIR'): 0.5633802816901409,
 ('FEMALE SHAPE-SHIFTER', 'BASHIR'): 0.6,
 ('BILBY', 'BASHIR'): 0.3333333333333333,
 ('KASIDY', 'BASHIR'): 0.6086956521739131,
 ('COM

In [173]:
rules[('BASHIR', 'GARAK')]

0.19377162629757785

In [174]:
rules[('GARAK', 'BASHIR')]

0.7044025157232704